In [1]:
from sage.matrix.berlekamp_massey import berlekamp_massey
from sage.arith.functions import LCM_list
import math
import random
import numpy as np
import time

def binary(n, length):
    n = Z(n)
    if length < n.nbits():
        raise ValueError(format(n,length))
    return '0' * (length-n.nbits()) + n.binary()


Z = IntegerRing()


def linear_recursion(S, alpha):
    degree = len(alpha)-1
    for i in range(len(S)-degree):
        RHS = 0
        for j in range(degree):
            RHS = (RHS + S[j+i]*alpha[j])%2
        if (S[degree + i] != RHS):
            return false
    return true


def minimal_S(S):
    # S can be S(1) to S(l)
    # takes S as string, but outputs minimal polynomial m(x) 
    s = "GF(2)"
    s += f"({S[0]})"
    for i in range(1,len(S)):
        s += f",{int(S[i])}"
        
    s = f"berlekamp_massey([{s}])"
    m = eval(s)
    return m


def S_from_SFy(SFy, i, M):
    # i <= l
    # M = l*l i.e, length of finite sequence SFy
    S = []
    for j in range(M):
        S.append(int(SFy[j][i]))
    return S  # a list of integers


def minimal_SFy(SFy, l):
    S_0 = S_from_SFy(SFy, 0, l**2)
    m = minimal_S(S_0)
    for i in range(1,l):
        if not (linear_recursion(S_from_SFy(SFy,i,l**2), m.list())):
            m_temp = minimal_S(S_from_SFy(SFy,i,l**2))
            R.<x> = PolynomialRing(GF(2))
            m = lcm(m, m_temp)
            

            return m # minimal polynomial of vector sequence S(F,y)

# calculation of local inverse

def LocalInverse_SFy(SFy,l):
    x = 0
    m = minimal_SFy(SFy,l)
    for i in range(l):
        x = x + sage.rings.integer.Integer(LocalInverse_S(S_from_SFy(SFy,i,l**2), m)) * (2**i)
    return x


def LocalInverse_S(S,m):
    degree = len(m.list()) - 1
    m_coeff = m.list()
    first_term = S[degree - 1]
    second_term = 0
    for i in (1..(degree-1)):
        second_term = (second_term + m_coeff[i]*S[i-1])%2
    
    localInverse_x = ((first_term - second_term)/m_coeff[0])%2
    return localInverse_x


def dec_to_bin(SFy, l):
    for i in range(len(SFy)):
        SFy[i] = binary(SFy[i],l)
        SFy[i] = SFy[i][::-1]
    return SFy


def func_e(e,n,SEy,l):
    for i in range (1,l*l):
        SEy.append(power_mod(SEy[i-1],e,n))
    
    return SEy


def func_d(c,n,SDy,l):
    for i in range (1,l*l):
        SDy.append(power_mod(c,SDy[i-1],n))
    
    return SDy



def P_Q_E(no_of_sets, bs_min, bs_max):
    primelist = []
    for i in range(2**bs_min - 1, 2**bs_max):
        if is_prime(i):
            primelist.append(i)
    
    p_q = random.sample(primelist, k=2*no_of_sets)
    

    
    triplet_set = []
    for i in range(no_of_sets):
        phi_n = (p_q[2*i] - 1)*(p_q[2*i + 1] - 1)
#         for j in range(phi_n//(10**(int(math.log(2**(2*bs_min),10) - 1))), 1, -1):
        for j in range(2,phi_n):
            if (gcd(j, phi_n) == 1):
                triplet_set.append([p_q[2*i], p_q[2*i + 1], j])
                break
    return triplet_set
        



        

In [2]:
def LocalInverse_E_Map(sample_size, p, q, e):
#     print(f"\n\n\n\n\nsample_size: {sample_size}, (p, q, e): {p, q, e}\nCompleted Iterations: ", end='')
    n = p*q
    c_list = random.sample(range(Z(1), Z(n-1)), k=sample_size)
    l = int(math.log(n,2) + 1)
    M = l*l
    y_x = []
    for i in range(sample_size):
#         print(i, end=',')
        
        SEy = [c_list[i]]
        SEy_original = func_e(e,n,SEy,l)
        SEy = SEy_original.copy()
        SEy = dec_to_bin(SEy,l)
#         print(SEy_original)
    
        try:
            LI_E = LocalInverse_SFy(SEy,l)
#         print("LcalInverse of D: ",LI_E)

        # SEy local inverse verification
            if (SEy_original[0] == power_mod(LI_E,e,n)):
#             print("SEy local inverse: Matched")
                y_x.append([c_list[i], LI_E])
                continue
            else:
#             print("SEy local inverse: Failed")
#             print(f"SEy_original[0]: {SEy_original[0]}, {type(SEy_original)}")
#             print(f"power_mod(c,LI_E,n): {power_mod(c,LI_E,n)}, {type(power_mod(c,LI_E,n))}")
                continue
        except:
            continue
    
    return y_x


def E_map_outcomes(sample_sizes, bs_min, bs_max):
    bit_length = bs_max
    table_parameters_E = []
    if (len(sample_sizes) == 1):  # list with one element which is list too
        no_of_sets = sample_sizes[0][0]
        sample_size = sample_sizes[0][1]
        triplet_set = P_Q_E(no_of_sets, bs_min, bs_max)
        for i in range(no_of_sets):
            print(i+1, end=', ')
            table_parameters_E_temp = [triplet_set[i][0], triplet_set[i][1], triplet_set[i][2], triplet_set[i][0] * triplet_set[i][1], bit_length, sample_size]
            y_x = LocalInverse_E_Map(sample_size, triplet_set[i][0], triplet_set[i][1], triplet_set[i][2])
            density_E = float(len(y_x))/sample_size
            table_parameters_E_temp.append(density_E)
            table_parameters_E.append(table_parameters_E_temp)
            
            
    else:
        no_of_sets = len(sample_sizes)
        triplet_set = P_Q_E(no_of_sets, bs_min, bs_max)
        for i in range(no_of_sets):
            print(i+1, end=', ')
            table_parameters_E_temp = [triplet_set[i][0], triplet_set[i][1], triplet_set[i][2], triplet_set[i][0] * triplet_set[i][1], bit_length, sample_sizes[i]]
            y_x = LocalInverse_E_Map(sample_sizes[i], triplet_set[i][0], triplet_set[i][1], triplet_set[i][2]) 
            density_E = float(len(y_x))/sample_sizes[i]
            table_parameters_E_temp.append(density_E)            
            table_parameters_E.append(table_parameters_E_temp)            

                
    print("\n\n","table_parameters_E", table_parameters_E,"\n\n")
    return None

In [ ]:
%%time
sample_sizes = [[5, 10000]]
bs_min = 9
bs_max = 10
E_map_outcomes(sample_sizes, bs_min, bs_max)

In [3]:
def LocalInverse_D_Map(sample_size, p, q, e):
#     print(f"\n\n\n\n\nsample_size: {sample_size}, (p, q, e): {p, q, e}\nCompleted Iterations: ", end='')
    n = p*q
    m_list = random.sample(range(Z(1), Z(n-1)), k=sample_size)
    l = int(math.log(n,2) + 1)
    M = l*l
    y_x = []
    for i in range(sample_size):
#         print(i, end=',')
        
        SDy = [m_list[i]]
        SDy_original = func_d(power_mod(SDy[0], e, n),n,SDy,l)
        SDy = SDy_original.copy()
        SDy = dec_to_bin(SDy,l)
#         print(SDy_original)
    
        try:
            LI_D = LocalInverse_SFy(SDy,l)
#         print("LcalInverse of D: ",LI_E)

        # SDy local inverse verification
            if (SDy_original[0] == power_mod(c,LI_D,n)):
                
#             print("SDy local inverse: Matched")
                y_x.append([m_list[i], LI_D])
                print(sample_size, p, q, e, LI_D)
                continue
            else:
#             print("SDy local inverse: Failed")
#             print(f"SDy_original[0]: {SDy_original[0]}, {type(SDy_original)}")
#             print(f"power_mod(c,LI_E,n): {power_mod(c,LI_E,n)}, {type(power_mod(c,LI_E,n))}")
                continue
        except:
            continue
    
    return y_x


def D_map_outcomes(sample_sizes, bs_min, bs_max):
#     going_out = []
    bit_length = bs_max
    table_parameters_D = []
    if (len(sample_sizes) == 1):  # list with one element which is list too
        no_of_sets = sample_sizes[0][0]
        sample_size = sample_sizes[0][1]
        triplet_set = P_Q_E(no_of_sets, bs_min, bs_max)
        for i in range(no_of_sets):
            print(i+1, end=', ')
            table_parameters_D_temp = [triplet_set[i][0], triplet_set[i][1], triplet_set[i][2], triplet_set[i][0] * triplet_set[i][1], bit_length, sample_size]
            y_x = LocalInverse_D_Map(sample_size, triplet_set[i][0], triplet_set[i][1], triplet_set[i][2])
            density_D = float(len(y_x))/sample_size
            table_parameters_D_temp.append(density_D)
            table_parameters_D.append(table_parameters_D_temp)
#             if (len(y_x) != 0):
#                 going_out.append([triplet_set[i][0], triplet_set[i][1], triplet_set[i][2], y_x])
# #                 print("\n{y : x}: ",y_x)
            
    else:
        no_of_sets = len(sample_sizes)
        triplet_set = P_Q_E(no_of_sets, bs_min, bs_max)
        for i in range(no_of_sets):
            print(i+1, end=', ')
            table_parameters_D_temp = [triplet_set[i][0], triplet_set[i][1], triplet_set[i][2], triplet_set[i][0] * triplet_set[i][1], bit_length, sample_sizes[i]]
            y_x = LocalInverse_D_Map(sample_sizes[i], triplet_set[i][0], triplet_set[i][1], triplet_set[i][2]) 
            density_D = float(len(y_x))/sample_sizes[i]
            table_parameters_D_temp.append(density_D)
            table_parameters_D.append(table_parameters_D_temp)            
#             if (len(y_x) != 0):
#                 going_out.append([triplet_set[i][0], triplet_set[i][1], triplet_set[i][2], y_x])
# #                 print("\n{y : x}: ",y_x)
                
    print("\n\n","table_parameters_D", table_parameters_D,"\n\n")
    return None


In [ ]:
sample_sizes = [[5, 10000]]
bs_min = 9
bs_max = 10
D_map_outcomes(sample_sizes, bs_min, bs_max)

In [ ]:
LocalInverse_E_Map(100, 541, 977, 7)

In [ ]:
%%time
for i in range(4):
    time.sleep(240)
    print("Iteration: ",i,"\n\n")
    sample_sizes = [[5, 10000]]
    bs_min = 9 + 2*i
    bs_max = 10 + 2*i
    E_map_outcomes(sample_sizes, bs_min, bs_max)
    time.sleep(240)
    D_map_outcomes(sample_sizes, bs_min, bs_max)
    
    

Iteration:  0 


1, 2, 3, 4, 5, 

 table_parameters_E [[641, 757, 11, 485237, 10, 10000, 0.0], [829, 811, 7, 672319, 10, 10000, 0.0912], [937, 761, 7, 713057, 10, 10000, 0.4678], [571, 953, 11, 544163, 10, 10000, 0.4798], [1009, 691, 11, 697219, 10, 10000, 0.4816]] 


1, 2, 3, 4, 5, 

 table_parameters_D [[887, 769, 5, 682103, 10, 10000, 0.0], [563, 907, 5, 510641, 10, 10000, 0.0], [967, 709, 5, 685603, 10, 10000, 0.0], [571, 743, 11, 424253, 10, 10000, 0.0], [673, 643, 5, 432739, 10, 10000, 0.0]] 


Iteration:  1 


1, 2, 3, 4, 5, 

 table_parameters_E [[3739, 2909, 5, 10876751, 12, 10000, 0.0005], [2287, 3359, 5, 7682033, 12, 10000, 0.0], [3347, 3331, 11, 11148857, 12, 10000, 0.0033], [3137, 3943, 5, 12369191, 12, 10000, 0.0691], [3251, 2927, 3, 9515677, 12, 10000, 0.0977]] 


1, 2, 3, 4, 5, 

 table_parameters_D [[2273, 2953, 5, 6712169, 12, 10000, 0.0], [2281, 3931, 7, 8966611, 12, 10000, 0.0], [3643, 2801, 11, 10204043, 12, 10000, 0.0], [2797, 3499, 5, 9786703, 12, 10000, 0.0], [4

In [5]:
sample_sizes = [[5, 40000]]
bs_min = 9
bs_max = 10
E_map_outcomes(sample_sizes, bs_min, bs_max)
time.sleep(240)
D_map_outcomes(sample_sizes, bs_min, bs_max)


1, 2, 3, 4, 5, 

 table_parameters_E [[599, 577, 5, 345623, 10, 40000, 0.0], [937, 787, 5, 737419, 10, 40000, 0.01865], [907, 643, 5, 583201, 10, 40000, 0.012025], [617, 727, 5, 448559, 10, 40000, 0.0], [631, 1009, 11, 636679, 10, 40000, 0.468975]] 


1, 2, 3, 4, 5, 

 table_parameters_D [[523, 691, 7, 361393, 10, 40000, 0.0], [877, 823, 5, 721771, 10, 40000, 0.0], [727, 739, 5, 537253, 10, 40000, 0.0], [769, 977, 5, 751313, 10, 40000, 0.0], [661, 809, 7, 534749, 10, 40000, 0.0]] 




In [6]:
sample_sizes = [[5, 10000]]
bs_min = 7
bs_max = 8
E_map_outcomes(sample_sizes, bs_min, bs_max)
time.sleep(240)
D_map_outcomes(sample_sizes, bs_min, bs_max)

1, 2, 3, 4, 5, 

 table_parameters_E [[163, 197, 5, 32111, 8, 10000, 0.0], [149, 251, 3, 37399, 8, 10000, 0.0319], [173, 127, 5, 21971, 8, 10000, 0.0], [211, 233, 11, 49163, 8, 10000, 0.559], [167, 241, 7, 40247, 8, 10000, 0.078]] 


1, 2, 3, 4, 5, 

 table_parameters_D [[163, 157, 5, 25591, 8, 10000, 0.0], [173, 137, 3, 23701, 8, 10000, 0.0], [211, 197, 11, 41567, 8, 10000, 0.0], [181, 193, 7, 34933, 8, 10000, 0.0], [139, 131, 7, 18209, 8, 10000, 0.0]] 




In [8]:
sample_sizes = [[5, 10000]]
bs_min = 9
bs_max = 10
E_map_outcomes(sample_sizes, bs_min, bs_max)
# time.sleep(240)
# D_map_outcomes(sample_sizes, bs_min, bs_max)

1, 2, 3, 4, 5, 

 table_parameters_E [[887, 683, 3, 605821, 10, 10000, 0.0013], [877, 809, 5, 709493, 10, 10000, 0.013], [937, 587, 5, 550019, 10, 10000, 0.003], [829, 653, 5, 541337, 10, 10000, 0.0236], [541, 743, 11, 401963, 10, 10000, 0.0]] 




In [4]:
sample_sizes = [[5, 10000]]
bs_min = 9
bs_max = 10
E_map_outcomes(sample_sizes, bs_min, bs_max)
    

1, 2, 3, 4, 5, 

 table_parameters_E [[839, 719, 3, 603241, 10, 10000, 0.0012], [709, 907, 5, 643063, 10, 10000, 0.0135], [1021, 991, 7, 1011811, 10, 10000, 0.1881], [593, 571, 7, 338603, 10, 10000, 0.0], [1019, 563, 3, 573697, 10, 10000, 0.0]] 




In [ ]:
sample_sizes = [[5, 10000]]
bs_min = 9
bs_max = 10
E_map_outcomes(sample_sizes, bs_min, bs_max)
    

iter: 1
1, 